In [ ]:
import os
import sys
import numpy as np
from numpy.linalg import inv
from scipy.signal import lsim
from pathlib import Path
from tqdm.notebook import tqdm
if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from pfcommon import parse_Jacobian_vars_file, parse_sparse_matrix_file
from filter_OU_inputs import fit_TF, filter_inputs

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
FIGURES_DIR = 'figures'

Load the data: the data file contains the transfer functions computed by `compute_spectra.py`.

In [ ]:
data_dir = Path('../data/IEEE39_GFM/general_load/Ta_6.9')
data_file = data_dir / '39 Bus New England System GFM_AC_TF_-4.0_6.0_500.npz'
assert data_file.exists(), f'{data_file}: no such file'
data = np.load(data_file, allow_pickle=True)
F = data['F']

Define the inputs and outputs among the variables available in the data file:

In [ ]:
input_names = ['Genstat_38']
all_input_names = data['input_names'].tolist()
assert all(name in all_input_names for name in input_names), "At least one input name missing from data file"
var_names = ['Grid-G 01.ElmSym.speed', 'Grid-Genstat_38.ElmGenstat.xspeed']
all_var_names = data['var_names'].tolist()
assert all(name in all_var_names for name in var_names), "At least one variable name missing from data file"
N_inputs = len(input_names)
N_vars = len(var_names)
input_idx = [all_input_names.index(name) for name in input_names]
var_idx = [all_var_names.index(name) for name in var_names]

Get the appropriate transfer functions:

In [ ]:
J, K = np.meshgrid(input_idx, var_idx, indexing='ij')
TF = data['TF'][:, J, K]

Fit the tranfer functions using the vector fitting algorithm and create the `lsim` objects:

In [ ]:
systems, _ = fit_TF(F, TF)

Define the inputs:

In [ ]:
srate = 200
dt = 1 / srate
tend = 600
time = np.r_[0 : tend + dt / 2 : dt]
N_samples = time.size
U = np.zeros((N_inputs, N_samples))
U[:, time < tend/2] = 0
U[:, time >= tend/2] = 0.2

Filter the inputs using the `lsim` objects above:

In [ ]:
Y = filter_inputs(systems, U, time)

Plot the results:

In [ ]:
ratios = [3 for _ in range(N_vars)]
ratios.append(1)
fig, ax = plt.subplots(N_vars + 1, 1, figsize=(5, (N_vars + 1) * 1.5), height_ratios=ratios)
for i in range(N_vars):
    ax[i].plot(time, Y[i, :], 'k')
    ax[i].set_ylabel(var_names[i])
ax[-1].plot(time, U[0], 'tab:red')
ax[-1].set_ylabel('Δpref [MW]')
ax[-1].set_xlabel('Time [s]')
sns.despine()
fig.tight_layout()